In [2]:
from keras.models import Sequential
import pandas as pd
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
import sys, re, glob
import numpy as np
from keras.layers import Input, Dense, Flatten

/home/nsuprotivniy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data

In [3]:
def load_data(folder, rng):
    X = []

    for i in range(rng):

        packets = list()
        max_node = 0
        max_time = 0

        for file in glob.glob("{}/output_{}/*.routes".format(folder, i)):

            handle = open(file, 'r')
            data = handle.read()
            handle.close()

            _nodes = re.split('\n\n', data);

            _nodes.pop()
            for node in _nodes:

                _strs = re.findall('(\d{1,3}(?:\.\d{1,3}){3})\s+\d{1,3}(?:\.\d{1,3}){3}\s+\d{1,3}(?:\.\d{1,3}){3}\s+\w+\s+-?\d+\.\d+\s+(\d+)', node)

                strings = list()
                for _str in _strs:
                    strings.append(dict(zip(('Destination', 'Hops'), _str)))

                header = re.findall('Node:\s+(\d+)\s+Time:\s+(\d+)', node)

                max_node = max(max_node, int(header[0][0]))
                max_time = max(max_time, int(header[0][1]))

                for _str in strings:
                    _str['Node'] = int(header[0][0])
                    _str['Time'] = int(header[0][1])
                    packets.append(_str)


        table = pd.DataFrame(packets)
        time_agg = table.groupby(["Node", "Destination"]).agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        node_agg = time_agg.groupby("Node").agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        aggregate = node_agg.agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        X.append(aggregate.values.flatten())

    X = np.array(X)
    X[np.isnan(X) | np.isinf(X)] = 0
    
    return X


In [5]:
malicious = load_data("../data/malicious", 100)
normal = load_data("../data/normal", 100)

# Model

In [6]:
SHAPE = malicious.shape[1]

In [7]:
OPTIMIZER = Adam()

In [8]:
model = Sequential()
model.add(Dense(2048, input_shape=(1000,)))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
model.add(Dense(SHAPE, activation='sigmoid'))
generator = model

generator.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)

In [9]:
model = Sequential()
model.add(Dense(10, input_shape=(SHAPE,)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(1, activation='sigmoid'))

discriminator = model

discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'] )

In [10]:
discriminator.trainable = False
model = Sequential()
model.add(generator)
model.add(discriminator)

stacked = model
stacked.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)

# Train

In [11]:
def train(malicious, normal, epochs=200, batch = 10):
    for cnt in range(epochs):
        
        ## train discriminator
        random_index =  np.random.randint(0, len(malicious) - batch)
        malicious_sample = malicious[random_index : random_index + batch]
        normal_sample = normal[random_index : random_index + batch]
        
        gen_noise = np.random.normal(0, 1, (batch,1000))
        syntetic = generator.predict(gen_noise)
        
        x_combined_batch = np.concatenate((malicious_sample, normal_sample, syntetic))
        y_combined_batch = np.concatenate((np.ones((batch, 1)), np.zeros((batch, 1)), np.zeros((batch, 1))))
        
        x_combined_batch =  np.clip(0.999, 0.001, x_combined_batch)
        
        d_loss = discriminator.train_on_batch(x_combined_batch, y_combined_batch)
    
        # train generator
        noise = np.random.normal(0, 1, (batch,1000))
        y_mislabled = np.ones((batch, 1))
        g_loss = stacked.train_on_batch(noise, y_mislabled)
        print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

In [12]:
train(malicious, normal)

/home/nsuprotivniy/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, [Discriminator :: d_loss: 0.924518], [ Generator :: loss: 0.588799]
epoch: 1, [Discriminator :: d_loss: 0.820769], [ Generator :: loss: 0.733534]
epoch: 2, [Discriminator :: d_loss: 0.705780], [ Generator :: loss: 0.827833]
epoch: 3, [Discriminator :: d_loss: 0.664807], [ Generator :: loss: 0.970326]
epoch: 4, [Discriminator :: d_loss: 0.681246], [ Generator :: loss: 0.996867]
epoch: 5, [Discriminator :: d_loss: 0.683362], [ Generator :: loss: 1.068430]
epoch: 6, [Discriminator :: d_loss: 0.722818], [ Generator :: loss: 1.119182]
epoch: 7, [Discriminator :: d_loss: 0.715819], [ Generator :: loss: 1.104407]
epoch: 8, [Discriminator :: d_loss: 0.703380], [ Generator :: loss: 1.104931]
epoch: 9, [Discriminator :: d_loss: 0.699541], [ Generator :: loss: 1.077045]
epoch: 10, [Discriminator :: d_loss: 0.683308], [ Generator :: loss: 0.969240]
epoch: 11, [Discriminator :: d_loss: 0.678748], [ Generator :: loss: 1.026698]
epoch: 12, [Discriminator :: d_loss: 0.698977], [ Generator ::

epoch: 108, [Discriminator :: d_loss: 0.515106], [ Generator :: loss: 2.249681]
epoch: 109, [Discriminator :: d_loss: 0.496225], [ Generator :: loss: 2.519892]
epoch: 110, [Discriminator :: d_loss: 0.544444], [ Generator :: loss: 2.552508]
epoch: 111, [Discriminator :: d_loss: 0.601502], [ Generator :: loss: 2.381139]
epoch: 112, [Discriminator :: d_loss: 0.527076], [ Generator :: loss: 3.219368]
epoch: 113, [Discriminator :: d_loss: 0.465126], [ Generator :: loss: 1.981304]
epoch: 114, [Discriminator :: d_loss: 0.598265], [ Generator :: loss: 2.178265]
epoch: 115, [Discriminator :: d_loss: 0.481088], [ Generator :: loss: 2.487565]
epoch: 116, [Discriminator :: d_loss: 0.522836], [ Generator :: loss: 2.350382]
epoch: 117, [Discriminator :: d_loss: 0.569982], [ Generator :: loss: 2.548879]
epoch: 118, [Discriminator :: d_loss: 0.531464], [ Generator :: loss: 2.035452]
epoch: 119, [Discriminator :: d_loss: 0.522917], [ Generator :: loss: 2.866101]
epoch: 120, [Discriminator :: d_loss: 0.

# Preidiction

In [13]:
X = np.clip(0.999, 0.001,np.concatenate((malicious, normal)))
y = np.concatenate((np.ones((100, 1)), np.zeros((100, 1))))
predict = discriminator.predict(X)

In [16]:
predict.flatten()

array([0.5297213 , 0.5260311 , 0.4552513 , 0.29114658, 0.519302  ,
       0.4838573 , 0.5261893 , 0.5146875 , 0.5209128 , 0.5449027 ,
       0.42899117, 0.48511767, 0.522623  , 0.33279607, 0.3898391 ,
       0.5271429 , 0.3746858 , 0.5239943 , 0.5393062 , 0.5429988 ,
       0.5301274 , 0.52228695, 0.35519844, 0.5498112 , 0.54116607,
       0.5407401 , 0.55221134, 0.53330934, 0.47260338, 0.54400885,
       0.5284057 , 0.43215245, 0.5159782 , 0.50539315, 0.44561556,
       0.46921915, 0.4521281 , 0.5282074 , 0.545667  , 0.53650296,
       0.5350041 , 0.37155318, 0.51021665, 0.52935493, 0.4151826 ,
       0.43897268, 0.53847873, 0.45460096, 0.3442491 , 0.40009287,
       0.53334224, 0.42708474, 0.44393903, 0.4421928 , 0.4656596 ,
       0.459181  , 0.42023203, 0.39834163, 0.48070705, 0.4718846 ,
       0.53320104, 0.534909  , 0.53406346, 0.39340568, 0.43232605,
       0.31525916, 0.48846745, 0.43380564, 0.55039626, 0.44425356,
       0.5407282 , 0.51611286, 0.54152393, 0.42778778, 0.47896

In [19]:
np.mean(y == (predict > 0.5))

0.68